In [1]:
import gc
import os
import talib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from os import getenv
from dotenv import load_dotenv

from strategy import long_short
from config import binance_client, prices_collection
from helper import vwap, check_decimals, current_position

load_dotenv()

client = binance_client()
prices = prices_collection()

In [20]:
last_date = prices.find().sort('date', -1).limit(1).next()['date']
print('Finding the last record in our DB')

new_data_query = client.get_historical_klines('ETHUSDT', '5m', str(last_date), limit=100)
print('Getting new data from Binance API (5m interval)')

new_data = pd.DataFrame(new_data_query[:-1], columns=['Open time', 'open', 'high', 'low', 'close', 'vol', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
new_data['date'] = pd.to_datetime(new_data['Close time'], unit='ms')
cols_to_drop = ['Open time', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
new_data.drop(cols_to_drop, axis=1, inplace=True)

cols = ['open', 'high', 'low', 'close', 'vol']
new_data[cols] = new_data[cols].apply(pd.to_numeric, errors='ignore', axis=1)

if len(new_data) > 0:
    print('Inserting new data into DB')
    prices_collection().insert_many(new_data.to_dict('records'))

df = pd.DataFrame(list(prices.find().sort('date', -1).limit(300)))
df.sort_values('date', inplace=True)

df.set_index('date', inplace=True)
close = df['close'].values
df['rsi'] = talib.RSI(close, timeperiod=14)
macd, macdsignal, df['macdhist'] = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
df['EMA9'] = talib.EMA(df.close, 9)
df['EMA55'] = talib.EMA(df.close, 55)
df['EMA200'] = talib.EMA(df.close, 200)

df['date'] = df.index
df.reset_index(drop=True, inplace=True)
df.fillna(method='ffill', inplace=True)

print('Calculating signals')
df['signal'] = df.apply(long_short, axis=1)

symbol = 'ETHUSD'
decimal = check_decimals(symbol)

Finding the last record in our DB
Getting new data from Binance API (5m interval)
Calculating signals


In [21]:
df

,_id,open,high,low,close,vol,rsi,macdhist,EMA9,EMA55,EMA200,date,signal
0,63a0e7062588a593b2ef413a,1171.83,1173.02,1169.40,1169.63,22.05294,NaN,NaN,NaN,NaN,NaN,2022-12-19 18:54:59.999,None
1,63a0e7062588a593b2ef413b,1168.73,1170.63,1167.68,1169.44,13.62850,NaN,NaN,NaN,NaN,NaN,2022-12-19 18:59:59.999,None
2,63a0e7062588a593b2ef413c,1168.67,1170.14,1168.14,1168.14,8.91380,NaN,NaN,NaN,NaN,NaN,2022-12-19 19:04:59.999,None
3,63a0e7062588a593b2ef413d,1168.72,1171.83,1168.71,1169.92,10.43588,NaN,NaN,NaN,NaN,NaN,2022-12-19 19:09:59.999,None
4,63a0e7062588a593b2ef413e,1170.36,1171.49,1169.95,1169.95,10.41475,NaN,NaN,NaN,NaN,NaN,2022-12-19 19:14:59.999,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,63a20d56dd85841ae6e0406b,1216.08,1216.80,1216.08,1216.80,1.39471,59.504101,0.287564,1215.759866,1213.378279,1204.406410,2022-12-20 19:29:59.999,BUY
296,63a20ea1dd85841ae6e0406e,1218.03,1218.03,1217.54,1217.54,0.35733,61.528766,0.297476,1216.115893,1213.526912,1204.537092,2022-12-20 19:34:59.999,BUY
297,63a20feddd85841ae6e04071,1217.51,1217.69,1216.50,1217.63,76.51695,61.779053,0.284875,1216.418714,1213.673451,1204.667370,2022-12-20 19:39:59.999,BUY
298,63a21139dd85841ae6e04074,1217.65,1218.03,1217.02,1217.49,20.45755,61.113004,0.243496,1216.632972,1213.809756,1204.794958,2022-12-20 19:44:59.999,BUY


In [ ]:
signal = df['signal'].iloc[-1]
position = current_position()

if signal == position:
    print('No change in position')
    
else:
    with open('current_position.txt', 'w') as f:
        f.write(signal)

    if signal == 'BUY':
        print(f'Signal went from: {position} to {signal}, flipping to long at {df["close"].iloc[-1]}')
        balance = client.get_asset_balance(asset='USD')
        quantity = round((float(balance['free']) * .95), decimal)
        order = client.create_order(symbol=symbol, side=signal, type='MARKET', quantity=quantity)
        print(order)

    if signal == 'SELL':
        print(f'Signal went from: {position} to {signal}, flipping to short at {df["close"].iloc[-1]}')
        balance = client.get_asset_balance(asset='ETH')
        quantity = round((float(balance['free']) * .95), decimal)
        order = client.create_order(symbol=symbol, side=signal, type='MARKET', quantity=quantity)
        print(order)

    if signal == 'None':
        print('No signal, holding position')
        pass

In [ ]:
df.to_csv('check.csv', index=False)

In [ ]:
df.plot(y=['close', 'EMA20'], x='date', figsize=(20, 10))